In [ ]:
import os
from dotenv import load_dotenv
import streamlit as st
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.utilities import WikipediaAPIWrapper

# .env 파일 로드
load_dotenv()

# ✅ OpenAI API Key 설정
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("❌ OpenAI API Key가 설정되지 않았습니다.")

# ✅ Wikipedia 검색 도구
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# ✅ DuckDuckGo 검색 도구
duckduckgo_tool = DuckDuckGoSearchRun()

# ✅ 웹사이트 스크래핑 도구
def scrape_website(url):
    try:
        loader = WebBaseLoader(url)
        docs = loader.load()
        return docs[0].page_content if docs else "❌ 웹사이트에서 콘텐츠를 찾을 수 없습니다."
    except Exception as e:
        return f"🚨 웹사이트 스크래핑 실패: {e}"

scrape_tool = Tool(
    name="Web Scraper",
    func=scrape_website,
    description="웹사이트에서 텍스트를 추출하는 도구",
)

# ✅ 조사 결과를 .txt 파일에 저장하는 도구
def save_to_txt(content, filename="research_results.txt"):
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        return f"✅ 결과가 {filename}에 저장되었습니다."
    except Exception as e:
        return f"🚨 파일 저장 실패: {e}"

save_tool = Tool(
    name="Save to TXT",
    func=save_to_txt,
    description="조사 결과를 .txt 파일로 저장하는 도구",
)

# ✅ LLM (GPT-4 모델) 설정
llm = ChatOpenAI(model="gpt-4-turbo", openai_api_key=openai_api_key)
# ✅ 챗봇을 위한 LangChain RetrievalQA 설정


# ✅ AI 에이전트 초기화
tools = [wikipedia_tool, duckduckgo_tool, scrape_tool, save_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# ✅ 사용자 입력 처리 및 실행
query = "Research about the XZ backdoor"
print(f"🔍 {query}에 대한 조사를 시작합니다...\n")
result = agent.run(query)

# ✅ 조사 결과 저장
save_to_txt(result, "research_XZ_backdoor.txt")

print("✅ 조사 완료! 결과가 'research_XZ_backdoor.txt'에 저장되었습니다.")

🔍 Research about the XZ backdoor에 대한 조사를 시작합니다...



> Entering new AgentExecutor chain...


NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}